In [1]:
import pyvisa
import matplotlib.pyplot as plt
import matplotlib

from nanodrivers.visa_drivers.anapico import ANAPICO
from nanodrivers.visa_drivers.dc import DC
from nanodrivers.visa_drivers.vna import VNA

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift
from scipy.io import savemat
from datetime import datetime, timedelta
import time

import os
Global_data_adress = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA"
exp_name = 'TWPA_noise'
adress = Global_data_adress+'\\'+exp_name
if not os.path.exists(adress):
  os.mkdir(adress)
  print("Folder %s created!" % adress)
else:
  print("Folder %s already exists" % adress)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise already exists


C:\Users\Demag\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
now = datetime.now()
path = adress +'\\'+ 'Full_BW_noise'
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\Full_BW_noise already exists


In [3]:
anapico = ANAPICO()
anapico.idn()

Connection exist: AnaPico AG,APMS20G,3C6-0B4F40003-0985,0.4.209



In [4]:
 anapico.set_off(1)

In [5]:
dc = DC()
dc.idn()

Connection exist: HEWLETT-PACKARD,33120A,0,7.0-5.0-1.0



In [6]:
dc.set_volt(0)

In [7]:
vna = VNA()
vna.idn()


KeyboardInterrupt



# Check if TWPA is connected

In [ ]:
%matplotlib inline

def get_ref_gain():
    anapico.set_off(1)
    mag, pha = vna.get_data()
    reference_mag_level = np.mean(mag)
    return reference_mag_level

def get_FP_sweep(dc_curr, freq_steps, pow_steps): 
    dc.set_volt(dc_curr, delay = 0)
    m_ref = get_ref_gain()
    power = np.linspace(low_power, high_power, pow_steps)
    frequency = np.linspace(low_freq, high_freq, freq_steps)
    anapico.set_on(1)
    
    vna.set_band(100)
    vna.set_nop(50)
    vna.set_power(0)
    vna.set_cw_freq(6e9)
    gain = np.empty((pow_steps,freq_steps))
    for i in range(pow_steps):
        print('Round {}/{} started'.format(i, pow_steps), end = '\r')
        for j in range(freq_steps):
            anapico.set_power(1, power[i])
            anapico.set_freq(1, frequency[j])
            mag, pha = vna.get_data()
            gain[i,j] = np.mean(mag)-m_ref

    return power, frequency, gain

low_freq = 11.1e+9
high_freq = 11.3e+9
low_power = 5
high_power = 10

start_time = datetime.now()
fig, ax = plt.subplots(figsize=(25,13))
power, freq, gain = get_FP_sweep(dc_curr=2.45, freq_steps = 20, pow_steps = 20) 
c = ax.pcolor(freq, power, gain, cmap='viridis')
fig.colorbar(c, ax=ax)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# Test OP|

In [ ]:
vna.set_lin()
vna.set_nop(200)
fr = vna.get_freq()

DC = 2
dc.set_volt(DC, delay = 0)
anapico.set_off(1)
mag_ref, pha_ref = vna.get_data()

pumpP = 11
anapico.set_power(1, pumpP)
pumpF = 11.64e+9
anapico.set_freq(1, pumpF)
anapico.set_on(1)
mag, pha = vna.get_data()

fig, ax = plt.subplots(1,2,figsize=(15,10))
ax[0].plot(fr, mag-mag_ref)
ax[1].plot(fr, pha-pha_ref)

# Full VNA sweep near point with max gain

find gain sweep point

In [ ]:
start = datetime.now() 
file = r'VNA_full_BW_sweep'+ start.strftime(r'%Y-%m-%d-%H-%M-%S') + '.mat'

file_name = path +'\\' + file
state = dict()

# Set DC param
DC = 2
dc.set_volt(DC, delay = 0)
state['DC'] = DC
state['DC_state'] = dc.dump()


# Set Anapico param

pumpP = 11
anapico.set_power(1, pumpP)
pumpF = 11.64e+9
anapico.set_freq(1, pumpF)
anapico.set_on(1)
state['anapico_state'] = anapico.dump()


# Set VNA param
nop  = 201
vna.set_nop(nop)
BW = 100
vna.set_band(BW)
avg = 20
VNA_power = 0
vna.set_power(VNA_power)

vna.set_cw()
state['nop'] = nop
state['BW'] = BW
state['avg'] = avg
state['VNA_power'] = VNA_power
state['VNA_state'] = vna.dump()

VNA_freqs = np.linspace(4, 8, 500)*1e9

mag = np.zeros((nop, avg, len(VNA_freqs)))
pha = np.zeros((nop, avg, len(VNA_freqs)))

state['mag'] = mag
state['pha'] = pha

for i in range(len(VNA_freqs)):
    
    start_time = datetime.now()
    print('New round {}/{} started at {}'.format(i, len(VNA_freqs), start_time.strftime(r'%Y/%m/%d %H:%M:%S')))
        
    vna.set_cw_freq(VNA_freqs[i])
    
    for j in range(avg):
        mag[:,j,i], pha[:,j,i] = vna.get_data()
        state['mag'] = mag
        state['pha'] = pha
        
    savemat(file_name, state)
    
    
    if i == 0: 
        duration_end = datetime.now()
        period = (duration_end-start_time).seconds
        
        print('Duration: {}. End of script in: {}'.format(duration_end - start_time, 
                                        timedelta(seconds=period*len(VNA_freqs))))
        

script_end_time = datetime.now()
print('Script finnished at {}'.format(script_end_time.strftime(r'%Y/%m/%d %H:%M:%S')))
dc.set_volt(0)
        

In [ ]:
start = datetime.now() 

path_el = r'C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\Line_calibration'
file_el = r'noise_empty_line'+ start.strftime(r'_%Y-%m-%d-%H-%M-%S') + '.mat'
file_name = path_el +'\\' + file_el
state = dict()

# Set VNA param
vna.set_cw()
nop  = 201
vna.set_nop(nop)
BW = 100
vna.set_band(BW)
avg = 5
VNA_power = 0
vna.set_power(VNA_power)

vna.set_cw()
state['nop'] = nop
state['BW'] = BW
state['avg'] = avg
state['VNA_power'] = VNA_power
state['VNA_state'] = vna.dump()

VNA_freqs = np.linspace(4, 8, 500)*1e9

mag = np.zeros((nop, avg, len(VNA_freqs)))
pha = np.zeros((nop, avg, len(VNA_freqs)))

state['mag'] = mag
state['pha'] = pha

for i in range(len(VNA_freqs)):
    
    start_time = datetime.now()
    print('New round {}/{} started at {}'.format(i, len(VNA_freqs), start_time.strftime(r'%Y/%m/%d %H:%M:%S')))
        
    vna.set_cw_freq(VNA_freqs[i])
    
    for j in range(avg):
        mag[:,j,i], pha[:,j,i] = vna.get_data()
        state['mag'] = mag
        state['pha'] = pha
        
    savemat(file_name, state)
    
    
    if i == 0: 
        duration_end = datetime.now()
        period = (duration_end-start_time).seconds
        
        print('Duration: {}. End of script in: {}'.format(duration_end - start_time, 
                                        timedelta(seconds=period*len(VNA_freqs))))
        

script_end_time = datetime.now()
print('Script finnished at {}'.format(script_end_time.strftime(r'%Y/%m/%d %H:%M:%S')))        

In [ ]:
# measure empty line 
dc.set_volt(0)
anapico.set_off(1)


vna.set_lin()
vna.set_power(-10)
vna.set_nop(10000)
vna.set_freq_start_stop(4*1e9,8*1e9,vna.get_nop())
mag_empty_line, pha_empty_line = vna.get_data()


In [ ]:
path_el = r'C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\Line_calibration'
file_el = r'empty_line_sweep_nop10000'+ start.strftime(r'_%Y-%m-%d-%H-%M-%S') + '.mat'
file_name = path_el +'\\' + file_el
state_el = dict()

state_el['mag'] = mag_empty_line
state_el['pha'] = pha_empty_line
state_el['vna'] = vna.dump()
state_el['temp'] = 8.8

savemat(file_name, state_el)

In [ ]:
%matplotlib qt
font = {'size'   : 15}
matplotlib.rc('font', **font)

fig, ax = plt.subplots(3,2,figsize=(20,13), sharex = True)
fig.suptitle("VNA_power = 0, pump_power = 11, DC = 2")
ax[0,0].set_title('magnitude gain')
ax[0,1].set_title('phase')
ax[1,0].set_title('magnitude variance')
ax[1,1].set_title('phase variance')
ax[2,0].set_title('magnitude gradient')
ax[1,1].set_title('phase gradient')
ax[0,0].grid()
ax[0,1].grid()
ax[1,0].grid()
ax[1,1].grid()
ax[2,0].grid()
ax[2,1].grid()

mag_mean = np.array([])
pha_mean = np.array([])
mag_var = np.array([])
pha_var = np.array([])

for i in range(len(VNA_freqs)):
    mm = np.reshape(mag[:,:,i], -1)
    pp = np.reshape(pha[:,:,i], -1)
    
    mag_mean = np.append(mag_mean, np.mean(mm)-mag_ref[i])
    pha_mean = np.append(pha_mean, np.mean(pp)-pha_ref[i])
    mag_var = np.append(mag_var, np.var(mm)/mag_mean[-1])
    pha_var = np.append(pha_var, np.var(pp))

ax[0,0].plot(VNA_freqs[1:]/1e9, mag_mean[1:])
ax[0,1].plot(VNA_freqs[1:]/1e9, pha_mean[1:])

ax[1,0].plot(VNA_freqs[1:]/1e9, mag_var[1:])
ax[1,1].plot(VNA_freqs[1:]/1e9, pha_var[1:])

ax[2,0].plot(VNA_freqs[1:]/1e9, np.abs(np.gradient(mag_mean[1:])))
ax[2,1].plot(VNA_freqs[1:]/1e9, np.abs(np.gradient(pha_mean[1:])))



# ax[0].errorbar(VNA_freqs, np.mean(mag[i],axis = 0), yerr=np.var(mag[i],axis = 0), fmt="r--o", ecolor = "black")
# ax[1].errorbar(VNA_freqs, np.mean(pha[i],axis = 0), yerr=np.var(pha[i],axis = 0), fmt="r--o", ecolor = "black")



In [ ]:
import os
import scipy.io as sio
initial_folder_empty_line = r'C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\Line_calibration'

files = os.listdir(initial_folder_empty_line)
for i in range(len(files)):
    if ".fig" in files[i]: pass
    else: 
        print(r"[%d]" %(i), '----------->', files[i]) 

In [ ]:
file_empty_line = files[0]
mat = sio.loadmat(initial_folder_empty_line+"\\"+file_empty_line)
mat.keys()

In [ ]:
mag_ref_line = mat['mag']
pha_ref_line = mat['pha']
freq_ref_line = mat['freq'][0]

len(mag_ref_line)
len(freq_ref_line)

# Plot line and TWPA on the same plot

In [ ]:
import os
import scipy.io as sio
initial_folder_empty_line = r'C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\Line_calibration'

files = os.listdir(initial_folder_empty_line)
for i in range(len(files)):
    if ".fig" in files[i]: pass
    else: 
        print(r"[%d]" %(i), '----------->', files[i]) 

In [ ]:
file_empty_line = files[5]
mat = sio.loadmat(initial_folder_empty_line+"\\"+file_empty_line)
mat.keys()

In [ ]:
mag_empty_line = mat['mag']
pha_empty_line = mat['pha']
freq_empty_line = np.linspace(4, 8, 500)

In [ ]:
import os
import scipy.io as sio
initial_folder_empty_line = r'C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\Full_BW_noise'

files = os.listdir(initial_folder_empty_line)
for i in range(len(files)):
    if ".fig" in files[i]: pass
    else: 
        print(r"[%d]" %(i), '----------->', files[i]) 

In [ ]:
file_empty_line = files[2]
mat = sio.loadmat(initial_folder_empty_line+"\\"+file_empty_line)
mat.keys()

In [ ]:
mag_TWPA = mat['mag']
pha_TWPA = mat['pha']
shape(mag_TWPA)

In [ ]:
%matplotlib qt
font = {'size'   : 15}
matplotlib.rc('font', **font)

fig, ax = plt.subplots(3,2,figsize=(20,13), sharex = True)
# fig.suptitle("VNA_power = 0, pump_power = 11, DC = 2")

ax[0,0].set_title('magnitude gain')
ax[0,1].set_title('phase')
ax[1,0].set_title('magnitude variance (normalized by gain)')
ax[1,1].set_title('phase variance')
ax[2,0].set_title('magnitude gradient')
ax[2,1].set_title('phase gradient')
ax[0,0].grid()
ax[0,1].grid()
ax[1,0].grid()
ax[1,1].grid()
ax[2,0].grid()
ax[2,1].grid()

mag_mean = np.array([])
pha_mean = np.array([])
mag_var = np.array([])
pha_var = np.array([])

for i in range(len(VNA_freqs)):
    mm = np.reshape(mag_empty_line[:,:,i], -1)
    pp = np.reshape(pha_empty_line[:,:,i], -1)
    
    mag_mean = np.append(mag_mean, np.mean(mm))
    pha_mean = np.append(pha_mean, np.mean(pp))
    mag_var = np.append(mag_var, np.var(mm))
    pha_var = np.append(pha_var, np.var(pp))

ax[0,0].plot(VNA_freqs[1:]/1e9, mag_mean[1:])
ax[0,1].plot(VNA_freqs[1:]/1e9, pha_mean[1:])

ax[1,0].plot(VNA_freqs[1:]/1e9, mag_var[1:])
ax[1,1].plot(VNA_freqs[1:]/1e9, pha_var[1:])

ax[2,0].plot(VNA_freqs[1:]/1e9, np.abs(np.gradient(mag_mean[1:])))
ax[2,1].plot(VNA_freqs[1:]/1e9, np.abs(np.gradient(pha_mean[1:])))


mag_mean = np.array([])
pha_mean = np.array([])
mag_var = np.array([])
pha_var = np.array([])

for i in range(len(VNA_freqs)):
    mm = np.reshape(mag_TWPA[:,:,i], -1)
    pp = np.reshape(pha_TWPA[:,:,i], -1)
    
    mag_mean = np.append(mag_mean, np.mean(mm)-mag_ref[i])
    pha_mean = np.append(pha_mean, np.mean(pp)-pha_ref[i])
    mag_var = np.append(mag_var, np.var(mm)/mag_mean[-1])
    pha_var = np.append(pha_var, np.var(pp))

ax[0,0].plot(VNA_freqs[1:]/1e9, mag_mean[1:])
ax[0,1].plot(VNA_freqs[1:]/1e9, pha_mean[1:])

ax[1,0].plot(VNA_freqs[1:]/1e9, mag_var[1:])
ax[1,1].plot(VNA_freqs[1:]/1e9, pha_var[1:])

ax[2,0].plot(VNA_freqs[1:]/1e9, np.abs(np.gradient(mag_mean[1:])))
ax[2,1].plot(VNA_freqs[1:]/1e9, np.abs(np.gradient(pha_mean[1:])))

